In [137]:
import os
import numpy
import torch
import time as timer
from PIL import Image
import numpy as np
import torch
from torch import nn
from torch import optim
from skimage.util import random_noise

from torchvision import datasets
import torchvision.transforms as T
import torchvision.utils as vutils

import res.viz_utils as vu
from res.plot_lib import set_default

set_default()
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset,DataLoader
from torch.nn import Module, Sequential, Conv2d, ConvTranspose2d, LeakyReLU, BatchNorm2d, ReLU, Tanh, Sigmoid, BCELoss
dev = 'cuda:0' if torch.cuda.is_available() == True else 'cpu'
device = torch.device(dev)
BATCH_SIZE = 64
NUM_EPOCH = 500
IMAGE_SIZE = 64
NUM_WORKERS = 2
DAY = '12may_newest'

In [136]:
!pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable


In [81]:
ls

AttentionGAN/            DCGAN/                    train_net_generator.ipynb
cyborg_images_32x32.npz  Human_Cyborg_Data/        Untitled.ipynb
cyborg_images_64x64.npz  instancenormalization.py
CycleGAN/                res/


In [53]:
def init_weights(m):
    if type(m) == ConvTranspose2d:
        nn.init.normal_(m.weight, 0.0, 0.02)
    elif type(m) == BatchNorm2d:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.constant_(m.bias, 0)

In [54]:
path_gambar = "Human_Cyborg_Data"
files_gambar = os.listdir(path_gambar)

In [55]:
path_gambar

'Human_Cyborg_Data'

In [58]:
def transformimg(file,dir_data):
    img = Image.open(dir_data+'/'+file)
    img = img.convert('RGB')
    img = img.resize((64,64))
    img = np.asarray(img)/255
    X_train = np.array(img)
    return X_train

In [57]:
file_jpg = [file for file in files_gambar if file[-4:] == ".jpg"]

In [122]:
X = []
y = []
for file in file_jpg:
    arr_jpg = transformimg(file,path_gambar)
    X.append(arr_jpg)
    y.append(int(file[0])) 
X = np.array(X)
y = np.array(y)
X_train = torch.from_numpy(X)
label = torch.from_numpy(y)

In [116]:
X_train.shape

torch.Size([37, 64, 64, 3])

In [120]:
y.shape

(37,)

In [123]:
X_train = np.transpose( # imp step to convert image size from (7312, 32,32,3) to (7312, 3,32,32)
    X_train, # imp step to convert double -&gt; float (by default numpy input uses double as data type)
    (0,3,1,2) # tuple to describe how to rearrange the dimensions
    )
X_train.shape

torch.Size([37, 3, 64, 64])

In [124]:
dataset = TensorDataset(X_train, label)
data_loader = DataLoader(dataset, batch_size = 16, shuffle = True)

In [ ]:
for batch_idx, (X,label) in enumerate(data_loader):
    print((X,label))

In [144]:
class Generator(Module):
    def __init__(self,ngpu=0):
 
        # calling constructor of parent class
        super().__init__()
        self.ngpu=ngpu
 
        self.gen = Sequential(
          ConvTranspose2d(in_channels = 100, out_channels = 512 , kernel_size = 4, stride = 1, padding = 0, bias = False),
          # the output from the above will be b_size ,512, 4,4
          BatchNorm2d(num_features = 512), # From an input of size (b_size, C, H, W), pick num_features = C
          ReLU(inplace = True),
 
          ConvTranspose2d(in_channels = 512, out_channels = 256 , kernel_size = 4, stride = 2, padding = 1, bias = False),
          # the output from the above will be b_size ,256, 8,8
          BatchNorm2d(num_features = 256),
          ReLU(inplace = True),
 
          ConvTranspose2d(in_channels = 256, out_channels = 128 , kernel_size = 4, stride = 2, padding = 1, bias = False),
          # the output from the above will be b_size ,128, 16,16
          BatchNorm2d(num_features = 128),
          ReLU(inplace = True),
          
          ConvTranspose2d(in_channels = 128, out_channels = 64 , kernel_size = 4, stride = 2, padding = 1, bias = False),
          # the output from the above will be b_size ,128, 16,16
          BatchNorm2d(num_features = 64),
          ReLU(inplace = True),  
 
          ConvTranspose2d(in_channels = 64, out_channels = 3 , kernel_size = 4, stride = 2, padding = 1, bias = False),
          # the output from the above will be b_size ,3, 32,32
          Tanh()
         
        )
 
    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.gen, input, range(self.ngpu))
        else:
            output = self.gen(input)
        return output
    
# class ExampleGenerator(Module):
 
#       def __init__(self):
#           # calling constructor of parent class
#           super().__init__()
#           self.gen = Sequential(
#               ConvTranspose2d(
#               in_channels = 100, 
#               out_channels = 512 , 
#               kernel_size = 4, 
#               stride = 1, 
#               padding = 0, 
#               bias = False)
#               )
 
#       def forward(self, input):
#            return self.gen(input)
 
# # defining class object
# ex_gen = ExampleGenerator()
 
# # defining random input for the model: b_size = 2 here
# t = torch.randn(2, 100, 1, 1)
 
# # checking the shape of the output from model
# ex_gen(t).shape

In [145]:
class Discriminator(Module):
    def __init__(self,ngpu=0):
 
        super().__init__()
        self.ngpu = ngpu
        self.dis = Sequential(
 
            # input is (3, 32, 32)
            Conv2d(in_channels = 3, out_channels = 64, kernel_size = 4, stride = 2, padding = 1, bias=False),
            # ouput from above layer is b_size, 32, 16, 16
            LeakyReLU(0.2, inplace=True),
 
            Conv2d(in_channels = 64, out_channels = 64*2, kernel_size = 4, stride = 2, padding = 1, bias=False),
            # ouput from above layer is b_size, 32*2, 8, 8
            BatchNorm2d(64 * 2),
            LeakyReLU(0.2, inplace=True),
 
            Conv2d(in_channels = 64*2, out_channels = 64*4, kernel_size = 4, stride = 2, padding = 1, bias=False),
            # ouput from above layer is b_size, 32*4, 4, 4
            BatchNorm2d(64 * 4),
            LeakyReLU(0.2, inplace=True),
 
            Conv2d(in_channels = 64*4, out_channels = 64*8, kernel_size = 4, stride = 2, padding = 1, bias=False),
            # ouput from above layer is b_size, 256, 2, 2
            # NOTE: spatial size of this layer is 2x2, hence in the final layer, the kernel size must be 2 instead (or smaller than) 4
            BatchNorm2d(64 * 8),
            LeakyReLU(0.2, inplace=True),
 
            Conv2d(in_channels = 64*8, out_channels = 1, kernel_size = 4, stride = 1, padding = 0, bias=False),
            # ouput from above layer is b_size, 1, 1, 1
            Sigmoid()
        )
     
    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.dis, input, range(self.ngpu))
        else:
            output = self.dis(input)
        return output.view(-1, 1).squeeze(1) 


In [146]:
def init_weights(m):
    if type(m) == ConvTranspose2d:
        nn.init.normal_(m.weight, 0.0, 0.02)
    elif type(m) == BatchNorm2d:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.constant_(m.bias, 0)
pretrained_dis_model_path = 'DCGAN/output/ckpt_output/dcgan_dis_z100_ep500_12may_newest.pth'
pretrained_gen_model_path = 'DCGAN/output/ckpt_output/dcgan_gen_z100_ep500_12may_newest.pth'
netG = Generator(ngpu=2).to(device)
netD = Discriminator(ngpu=2).to(device)
netD.apply(init_weights)
netG.apply(init_weights)
netD.load_state_dict(torch.load(pretrained_dis_model_path))
netG.load_state_dict(torch.load(pretrained_gen_model_path))
print(netG)
print(netD)
optimizerD = optim.Adam(netD.parameters(), lr=2e-4, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=2e-4, betas=(0.5, 0.999))
criterion = nn.BCELoss()

Generator(
  (gen): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)
D

In [147]:
gname = "dcgan_gen" 
MODEL_DIR = 'DCGAN/output/ckpt_output/'
SAMPLE_DIR = 'DCGAN/output/image_gen'
fixed_noise = torch.randn(64, 100, 1, 1, device=device)
real_label = 1
fake_label = 0
gen_path = os.path.join(MODEL_DIR, f"{gname}_z{100}_ep{NUM_EPOCH}_{DAY}_newest.pth")

dname = "dcgan_dis" 
dis_path = os.path.join(MODEL_DIR, f"{dname}_z{100}_ep{NUM_EPOCH}_{DAY}_newest.pth")
for epoch in range(NUM_EPOCH):
    
    for batch_idx, (X, _) in enumerate(data_loader):
        start_t = timer.time()
        ############################
        #  Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################

        # Train with real image
        netD.zero_grad()
        real_cpu = X
        real_cpu = real_cpu.float()
        real_cpu = real_cpu.to(device)
        batch_size = real_cpu.size(0)
        label = torch.full(
            (batch_size,), real_label,
            dtype=real_cpu.dtype, 
            device=device
        )

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # Train with fake image
        noise = torch.randn(batch_size, 100, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # Update G network: minimize log(D(x)) + log(1 - D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label) # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        elapsed_t = timer.time() - start_t

        if batch_idx % 100 == 0:
            print(f'[{epoch+1}/{NUM_EPOCH}] [{batch_idx} / {len(data_loader)}] Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f} D(x): {D_x:.4f} D(G(z)): {D_G_z1:.4f} / {D_G_z2:.4f} Elapsed: {elapsed_t:.2f} secs')
            vutils.save_image(real_cpu, f'{SAMPLE_DIR}/real_samples_epoch-{epoch+1}_batch-{batch_idx}.png', normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(), f'{SAMPLE_DIR}/fake_samples_epoch-{epoch+1}_batch-{batch_idx}.png', normalize=True)

            # Checkpointing
            torch.save(netD.state_dict(), dis_path)
            print(f" --- Discriminator model stored in {dis_path} ---")

            torch.save(netG.state_dict(), gen_path)
            print(f" --- Generator model stored in {gen_path} ---") 

[1/500] [0 / 3] Loss_D: 0.3786 Loss_G: 10.9816 D(x): 0.8010 D(G(z)): 0.0000 / 0.0021 Elapsed: 0.92 secs
 --- Discriminator model stored in DCGAN/output/ckpt_output/dcgan_dis_z100_ep500_12may_newest_newest.pth ---
 --- Generator model stored in DCGAN/output/ckpt_output/dcgan_gen_z100_ep500_12may_newest_newest.pth ---
[2/500] [0 / 3] Loss_D: 0.0019 Loss_G: 9.9424 D(x): 0.9998 D(G(z)): 0.0017 / 0.0011 Elapsed: 0.29 secs
 --- Discriminator model stored in DCGAN/output/ckpt_output/dcgan_dis_z100_ep500_12may_newest_newest.pth ---
 --- Generator model stored in DCGAN/output/ckpt_output/dcgan_gen_z100_ep500_12may_newest_newest.pth ---
[3/500] [0 / 3] Loss_D: 0.0074 Loss_G: 8.3174 D(x): 0.9981 D(G(z)): 0.0055 / 0.0008 Elapsed: 0.28 secs
 --- Discriminator model stored in DCGAN/output/ckpt_output/dcgan_dis_z100_ep500_12may_newest_newest.pth ---
 --- Generator model stored in DCGAN/output/ckpt_output/dcgan_gen_z100_ep500_12may_newest_newest.pth ---
[4/500] [0 / 3] Loss_D: 0.0041 Loss_G: 8.9623 D(

In [ ]:
UAS_Deep_Learning/DCGAN/output/image_gen